Copyright (c) 2017-2019 [Serpent-Tools developers team](https://github.com/CORE-GATECH-GROUP/serpent-tools/graphs/contributors), GTRC

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Data files are not included with the python package, but can be downloaded from the [GitHub repository](https://github.com/CORE-GATECH-GROUP/serpent-tools). For this tutorial, the files are placed in the directory identified with the ``SERPENT_TOOLS_DATA`` environment variable.

In [1]:
import os
mdxFile = os.path.join(
    os.environ["SERPENT_TOOLS_DATA"],
    "ref_mdx0.m",
)

# `MicroXSReader`

## Basic Operation

This notebook demonstrates the capabilities of the [`serpentTools`](https://github.com/CORE-GATECH-GROUP/serpent-tools) in regards to reading group micro cross-section files. SERPENT [1] produces a [micro depletion file](http://serpent.vtt.fi/mediawiki/index.php/Description_of_output_files#Micro_depletion_output), containing independent and cumulative fission yields as well as group cross-sections for the isotopes and reactions defined by the user.
The `MicroXSReader` is capable of reading this file, and storing the data directly on the reader. The reader has two methods to retrieve the data and ease the analysis. Note: in order to obtain the micro depletion files, the user must set the `mdep` card in the input [file](http://serpent.vtt.fi/mediawiki/index.php/Input_syntax_manual#set_mdep).

In [2]:
import serpentTools

In [3]:
mdx = serpentTools.read(mdxFile)

The fission yields read in from the file are stored in the `nfy` dictionary, where the keys represent a specific (parent, energy) pair and the corresponding values is a dictionary with fission products ids and corresponding fission yield values. 

In [4]:
# All the (parent, energy) pairs can be obtained by using '.keys()'
pairs = mdx.nfy.keys()
list(pairs)[0:5] # list only the first five pairs

[(902270, 2.53e-08),
 (902280, 2.53e-08),
 (902280, 0.5),
 (902280, 14.0),
 (902290, 2.53e-08)]

Each pair represents the isotope undergoing fission and the impending neutron energy in MeV.

In [5]:
pair = list(pairs)[0] # obtain the first (isotope, energy) pair
print('Isotope= {: 1.0f}'.format(pair[0]))
print('Energy= {} MeV'.format(pair[1]))

Isotope=  902270
Energy= 2.53e-08 MeV


The results for each pair are dictionaries that contain three fields:

`fissProd` list of fission products ids

`indYield` corresponding list of independent fission yields

`cumYield` corresponding list of cumulative fission yields

In [6]:
# Obtain the keys in the nfy dictionary
mdx.nfy[pair].keys()

dict_keys(['fissProd', 'indYield', 'cumYield'])

In [7]:
# Print only the five first fission products
print(mdx.nfy[pair]['fissProd'][0:5])

[250660. 250670. 250680. 260660. 260670.]


In [8]:
# Print only the five first fission independent yields
print(mdx.nfy[pair]['indYield'][0:5])

[6.97001e-13 1.35000e-13 1.01000e-14 2.57000e-10 1.13000e-10]


In [9]:
# Print only the five first fission cumulative yields
print(mdx.nfy[pair]['cumYield'][0:5])

[6.97001e-13 1.35000e-13 1.01000e-14 2.58000e-10 1.13000e-10]


Fluxes ratios and uncertainties are stored in the `fluxRatio` and `fluxUnc` dictionaries, where the keys represent a specific universe and the corresponding values are group fluxes values.

In [10]:
# obtain the universes
print(mdx.fluxRatio.keys())

dict_keys(['0'])


Cross sections and their uncertainties are stored in the `xsVal` and `xsUnc` dictionaries, where the keys represent a specific universe and the corresponding values are dictionaries.

In [11]:
# The keys within the nested dictionary describe the isotope, reaction and special flag
print(mdx.xsVal['0'].keys())

dict_keys([MicroXSTuple(zai=10010, mt=102, metastable=0), MicroXSTuple(zai=982490, mt=18, metastable=0), MicroXSTuple(zai=982510, mt=102, metastable=0), MicroXSTuple(zai=982510, mt=16, metastable=0), MicroXSTuple(zai=982510, mt=17, metastable=0), MicroXSTuple(zai=982510, mt=18, metastable=0), MicroXSTuple(zai=40090, mt=107, metastable=0)])


Each key has three entries (isotope, reaction, flag)

`isotope` ID of the isotope (ZZAAA0/1), int or float

`reaction` MT [reaction](http://serpent.vtt.fi/mediawiki/index.php/ENDF_reaction_MT%27s_and_macroscopic_reaction_numbers), e.g., 102 (n,gamma)

`flag` special flag to describe isomeric state or fission yield distribution number

For each such key (isotope, reaction, flag) the `xsVal` and `xsVal` store the group-wise flux values and uncertainties respectively.  

In [12]:
val = mdx.xsVal['0']
unc = mdx.xsUnc['0']

In [13]:
# Print flux values
print(val[(10010, 102, 0)])

[3.09753e-05 3.33901e-05 3.57054e-05 3.70926e-05 3.61049e-05 3.39464e-05
 3.39767e-05 3.98315e-05 5.38962e-05 7.96923e-05 1.18509e-04 1.73915e-04
 2.54571e-04 3.38540e-04 4.52415e-04 5.98190e-04 7.69483e-04 1.04855e-03
 1.31149e-03 1.67790e-03 2.15195e-03 2.70125e-03 3.44635e-03 5.04611e-03]


In [14]:
# Print flux uncertainties
print(unc[(10010, 102, 0)])

[1.1e-04 2.0e-05 1.0e-05 0.0e+00 0.0e+00 0.0e+00 0.0e+00 1.0e-05 1.0e-05
 2.0e-05 2.0e-05 2.0e-05 2.0e-05 1.0e-05 1.0e-05 2.0e-05 2.0e-05 3.0e-05
 2.0e-05 3.0e-05 4.0e-05 5.0e-05 1.7e-04 6.9e-04]


## Data Retrieval
The `MicroXSReader` object has two `get` methods. 

`getFY` method obtains the independent and cumulative fission yields for a specific parent (ZZAAA0/1), daughter (ZZAAA0/1), neutron energy (MeV). If no parent or daaughter is found, the method raises an exception. The method also has a special flag that indicates whether the user wants to obtain the value corresponding to the nearest energy. 

`getXS` method to obtain the group-wise cross-sections for a specific universe, isotope and reaction. 

In [15]:
indYield, cumYield = mdx.getFY(parent=922350, energy=2.53e-08, daughter=541350 )
print('Independent yield = {}'.format(indYield))
print('Cumulative yield = {}'.format(cumYield))

Independent yield = 0.000785125
Cumulative yield = 0.065385


By default, the method includes a flag that allows to obtain the values for the closest energy defined by the user.

In [16]:
indYield, cumYield = mdx.getFY(parent=922350, energy=1e-06, daughter=541350 )
print('Independent yield = {}'.format(indYield))
print('Cumulative yield = {}'.format(cumYield))

Independent yield = 0.000785125
Cumulative yield = 0.065385


The user can set this boolean flag to False if only the values at existing energies are of interest.

In [17]:
indYield, cumYield = mdx.getFY(parent=922350, energy=2.53e-08, daughter=541350, flagEnergy=False )

`getXS` method is used to obtain the group cross-sections for a specific universe, isotope and reaction. The method returns the values and uncertainties. 

In [18]:
# Obtain the group cross-sections
vals, unc = mdx.getXS(universe='0', isotope=10010, reaction=102)

In [19]:
# Print group flux values
print(vals)

[3.09753e-05 3.33901e-05 3.57054e-05 3.70926e-05 3.61049e-05 3.39464e-05
 3.39767e-05 3.98315e-05 5.38962e-05 7.96923e-05 1.18509e-04 1.73915e-04
 2.54571e-04 3.38540e-04 4.52415e-04 5.98190e-04 7.69483e-04 1.04855e-03
 1.31149e-03 1.67790e-03 2.15195e-03 2.70125e-03 3.44635e-03 5.04611e-03]


In [20]:
# Print group flux uncertainties values
print(unc)

[1.1e-04 2.0e-05 1.0e-05 0.0e+00 0.0e+00 0.0e+00 0.0e+00 1.0e-05 1.0e-05
 2.0e-05 2.0e-05 2.0e-05 2.0e-05 1.0e-05 1.0e-05 2.0e-05 2.0e-05 3.0e-05
 2.0e-05 3.0e-05 4.0e-05 5.0e-05 1.7e-04 6.9e-04]


The method includes a special flag `isomeric`, which is set to zero by default. 
The special flag either describes the isomeric state or fission yield distribution number.

In [21]:
# Example of how to use the isomeric flag
vals, unc = mdx.getXS(universe='0', isotope=10010, reaction=102, isomeric=0)

If the universe exist, but the isotope or reaction do not exist, the method raises an error.

## Settings
The `MicroXSReader` also has a collection of settings to control what data is stored. If none of these settings are modified, the default is to store all the data from the output file.

In [22]:
from serpentTools.settings import rc

In [23]:
rc['microxs.getFY'] = False # True/False only
rc['microxs.getXS'] = True # True/False only
rc['microxs.getFlx'] = True # True/False only

`microxs.getFY`: True or False, store fission yields

`microxs.getXS`: True or False, store group cross-sections and uncertainties

`microxs.getFlx`: True or False, store flux ratios and uncertainties

In [24]:
mdx = serpentTools.read(mdxFile)

In [25]:
# fission yields are not stored on the reader
mdx.nfy.keys()

dict_keys([])

## Conclusion
The `MicroXSReader` is capable of reading and storing all the data from the SERPENT micro depletion file. Fission yields, cross-sections and flux ratios are stored on the reader. The reader also includes two methods `getFY` and `getXS` to retrieve the data. Use of the `rc` settings control object allows increased control over the data selected from the output file.

## References
1. J. Leppänen, M. Pusa, T. Viitanen, V. Valtavirta, and T. Kaltiaisenaho. "The Serpent Monte Carlo code: Status, development and applications in 2013." Ann. Nucl. Energy, [82 (2015) 142-150](https://www.sciencedirect.com/science/article/pii/S0306454914004095)